# SYCL devices


## Buffer and Accesors


In [101]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>
#include <iostream>

int main() {

    sycl::queue myQueue;                         // The queue, Q, is the object that
                                                 // submits the task to a device.
    int const N = 10;
    sycl::buffer<int> bufA{ sycl::range{N} };    // The buffer, A, is the memory used to
                                                 // transfer data between host and device.
    const int p = 4;
    std::cout << "Running on "
                  << myQueue.get_device().get_info<sycl::info::device::name>()
                  << "\n";

    myQueue.submit([&](sycl::handler& cgh) {      // The handler, h, is the object that contains
                                                  // the single_task function to be used.

        sycl::accessor accA(bufA, cgh);          // The accessor, A_acc, is the object that
                                                  // efficiently accesses the buffer elements.

            cgh.parallel_for(N, [=](auto i) {
                accA[i] = i * p; 
                });

        });

    sycl::host_accessor result(bufA);              // host_accessor is the object that allows
                                                   // the host to access the buffer memory.

    for (int i = 0; i < N ; i++)                   // Print output
        std::cout << result[i] << " "; std::cout << "\n";
    return 0;

}

Overwriting compute.cpp


In [99]:
%%writefile ./run-sycl.sh
#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [100]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

Running on Intel(R) Graphics [0x46a6]
0 4 8 12 16 20 24 28 32 36 


## Example 2:

In [98]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>
#include <iostream>

int main(int argc, char **argv) {
    
    const int N = 10;
    std::vector<int> vectorA(N);  // Vector A filled with 1s
    const int p = 4;
    
    {
        auto myQueue = sycl::queue{sycl::gpu_selector_v};
        std::cout << "Running on "
                  << myQueue.get_device().get_info<sycl::info::device::name>()
                  << "\n";

    
        auto bufA = sycl::buffer<int>{ vectorA.data(), vectorA.size()};
        //sycl::buffer<int> bufA {vectorA.data(),vectorA.size() };
        
        myQueue.submit([&](sycl::handler &cgh) {
        
            sycl::accessor accA { bufA, cgh, sycl::write_only, sycl::no_init};
            
            cgh.parallel_for(N, [=](auto i) {
                accA[i] = i * p; 
                });
        
            });
        myQueue.wait();
        
    }

    for (int i = 0; i < N; i++)  // Print output
        std::cout << vectorA[i] << " "; std::cout << "\n";

    return 0;
}


Overwriting compute.cpp


In [102]:
%%writefile ./run-sycl.sh
#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [103]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

Running on Intel(R) Graphics [0x46a6]
0 4 8 12 16 20 24 28 32 36 
